**Tridiagonal Matrices (Homework)**

**Alexandra Beikert, Pascal Huber, León-Alexander Hühn**

*1. Derive the iterative expressions for Gaussian elimination*

To derive the inverse $T^{-1}$ of the tridiagonal matrix $T$, we first set $t^{-1}_{ij} = \delta_{ij}$  
Starting from row k=1 to k=N, calculate (if steps contain invalid indices, ignore them)  
$ t^{-1}_{kj} = t^{-1}_{kj} / t_{kk} \qquad \text{for} \ j=k-1,k,k+1$  
$ t^{-1}_{k+1,k} = t^{-1}_{k+1,k} - t^{-1}_{kk} \cdot t_{k+1,k} / t_{kk}$  
$ t^{-1}_{k+1,k+1} = t^{-1}_{k+1,k+1} - t^{-1}_{k,k+1} \cdot t_{k+1,k} / t_{kk}$  
$ t^{-1}_{k-1,k} =  t^{-1}_{k-1,k} - t^{-1}_{kk} \cdot t_{k-1,k} / t_{kk}$  
$ t^{-1}_{k-1,k+1} =  t^{-1}_{k-1,k} - t^{-1}_{k,k+1} \cdot t_{k-1,k} / t_{kk}$  
To obtain the solution of the system of equations, calculate $T^{-1} \cdot \vec{y}$

*2. Derive the iterative expressions for backward substitution*

To derive the solution to $T \cdot \vec{x} = \vec{y}$, calculate for rows k=1 to k=N (ignore invalid indices as above):  
$ t_{kj} = t_{kj} / t_{kk} \qquad \text{for} \ j=k-1,k+1$  
$ y_k = y_k / t_{kk}$  
$ t_{kk} = t_{kk} / t_{kk} = 1$
$ t_{k+1,k} = t_{k+1,k} - t_{k+1,k} = 0$  
$ t_{k+1,k+1} = t_{k+1,k+1} - t_{k,k+1} \cdot t_{k+1,k}$  
$ y_{k+1} = y_{k+1} - t_{k+1,k} \cdot y_k$  
This will result in $[T, \vec{y}]$ being in upper triangular form. From there, the solution is given by  
Starting at i=N to i=1  
$ x_i = \frac{1}{t_{ii}} \left( y_i - \sum_{j > i} t_{ij}\cdot x_j \right)$

*3. Program a subroutine that, given the values $a_2...a_N$, $b_1...b_N$, $c_1...c_{N-1}$ and $y_1...y_N$, finds the solution vector given by $x_1,...,x_N$*

In [1]:
import numpy as np

In [2]:
# a,b,c,y are arrays containing the given information (see above)
# Notice the index shift to start at 0 rather than 1
def solve_bsub(a,b,c,y):
    N = len(b)
    T = generate_matrix(a,b,c)
    for k in range(0,N):
        if k-1 >= 0:
            T[k,k-1] = T[k,k-1] / T[k,k]
        if k+1 < N:
            T[k,k+1] = T[k,k+1] / T[k,k]
        y[k] = y[k] / T[k,k]
        T[k,k] = 1
        if k+1 < N:
            T[k+1,k+1] -= T[k,k+1] * T[k+1,k]
            y[k+1] -= T[k+1,k]*y[k]
            T[k+1,k] = 0
    x = [0 for z in range(0,N)]
    for i in reversed(range(0, N)):
        _sum = 0
        for j in reversed(range(0, N)):
            if j > i:
                _sum += T[i,j] * x[j]
        x[i] = 1 / T[i,i] * (y[i] - _sum)
    return np.matrix(x)

# Function for generating the numpy matrix according to our definition
def generate_matrix(a,b,c):
    rows = [[] for i in range(0,len(b))]
    for k in range(0,len(b)):
        row = [0 for i in range(0,len(b))]
        row[k] = b[k]
        if k-1 >=0:
            row[k-1] = a[k-1] # because a2 is the first value in the array
        if k+1 < len(b):
            row[k+1] = c[k]
        rows[k] = row
    return np.matrix(rows, dtype=np.float_)

*4. Take $N=10$, and set all $a$ values to $-1$, all $b$ values to $2$, all $c$ values to $-1$ and all $y$ values to 0.1. What is the solution for the $x_1...x_N$?*

In [3]:
a = [-1 for i in range(0,9)]
b = [2 for i in range(0,10)]
c = [-1 for i in range(0,9)]
y = [.1 for i in range(0,10)]
x = solve_bsub(a,b,c,y)
print(x)

[[0.5 0.9 1.2 1.4 1.5 1.5 1.4 1.2 0.9 0.5]]


*5. Put your solution $x_1 ... x_N$ back into the original matrix equation above and find how much the result deviates from the original right-hand-side $y_1 ... y_N$. Is this satisfactory?*

In [4]:
#we check whether our solution is correct by multiplying with the original matrix

M = [] 
M.append([2., -1., 0., 0., 0., 0., 0., 0., 0., 0.])
M.append([-1., 2., -1., 0., 0., 0., 0., 0., 0., 0.])
M.append([0., -1., 2., -1., 0., 0., 0., 0., 0., 0.])
M.append([0., 0., -1., 2., -1., 0., 0., 0., 0., 0.])
M.append([0., 0., 0., -1., 2., -1., 0., 0., 0., 0.])
M.append([0., 0., 0., 0., -1., 2., -1., 0., 0., 0.])
M.append([0., 0., 0., 0., 0., -1., 2., -1., 0., 0.])
M.append([0., 0., 0., 0., 0., 0., -1., 2., -1., 0.])
M.append([0., 0., 0., 0., 0., 0., 0., -1., 2., -1.])
M.append([0., 0., 0., 0., 0., 0., 0., 0., -1., 2.])

M = np.matrix(M)
x = x.transpose()
print("Mx =")
print(M*x)

Mx =
[[0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]]


Multiplying our solution with our original Matrix M delivers the exact solution we started with.